<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" align="left" src="https://i.creativecommons.org/l/by-nc-sa/4.0/80x15.png" /></a>&nbsp;| [Emmanuel Rachelson](https://personnel.isae-supaero.fr/emmanuel-rachelson?lang=en) | <a href="https://erachelson.github.io/RLclass_MVA/">https://erachelson.github.io/RLclass_MVA/</a>

<div style="font-size:22pt; line-height:25pt; font-weight:bold; text-align:center;">Chapter 3: Learning value functions</div>

<div class="alert alert-success">

**Learning outcomes**  
By the end of this chapter, you should be able to:
- explain approximate value iteration and recall its (non-)convergence properties
</div>

# Approximate dynamic programming

Let's take a step back on the contents of the previous chapter.

With the Bellman equation, we have a way to **characterize** $Q^*$. This characterization directly translates to the **Value Iteration** algorithm. In turn, once we know $Q^*$, we can deduce $\pi^*$.

That's all very nice, but is it applicable in practice, on real world examples? In particular, how does the computation of $Q^*$ scale with large state and action spaces?

<div class="alert alert-warning">
    
**Exercise**  
Do you recall the time complexity of a single iteration of value iteration in terms of $|S|$ and $|A|$?  
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>

$O(S^2 A)$
</details>

The curse of dimensionality makes the number of states and actions scale exponentially with the dimension of the state and action spaces. So exact computation of $Q^*$ quickly becomes intractable, as building $Q_{n+1} = T^*Q_n$ from $Q_n$ requires $|S|$ operations in every state-action pair (hence the complexity in the exercise above).

Instead, one can try to *approximate* the resolution of $T^* Q_n$ at each step of Value Iteration. This yields the Approximate Value Iteration algorithm.

<div class="alert alert-success">
    
**Approximate Value Iteration** is the algorithm that computes the sequence $Q_{n+1} = \mathcal{A} T^* Q_n$, where $\mathcal{A}$ is an approximation procedure.
</div>

Note in particular that when dealing with parametric functions $Q_\theta$, finding a minimizer of the loss
$L_n(\theta) = \| Q_\theta - T^* Q_n \|$
is such an approximation procedure.

Let us suppose that $\mathcal{A}$ is not a bad approximation procedure and that its approximation error is uniformly bounded, that is, 
$$\forall f \in \mathcal{F}(S\times A,\mathbb{R}), \ \| f-\mathcal{A}f \|_\infty \leq \epsilon.$$

The first important result is that Approximate Value Iteration **does not converge**. However, one can prove that $Q_n$ reaches a neighborhood of $Q^*$. Specifically, there exists $N$ such that for all $n\geq N$,
$$\| Q^* - Q_n \|_\infty \leq \frac{\epsilon}{1-\gamma}.$$

More importantly, let $\pi_n$ be the greedy policy with respect to $Q_n$, then:
$$\|Q^*-Q^{\pi_n}\|_\infty \leq \frac{2\gamma}{1-\gamma} \|Q^*-Q_n\|_\infty.$$

And consequently, for such $n\geq N$,
$$\|Q^*-Q^{\pi_n}\|_\infty \leq \frac{2\gamma\epsilon}{(1-\gamma)^2}.$$

So,
<div class="alert alert-success">

Approximate Value Iteration does not necessarily converge but reaches policies whose values are close to optimal.
</div>

These results are proven in the **[Neuro-dynamic programming](http://athenasc.com/ndpbook.html)** book by D. P. Bertsekas and J. Tsitsiklis (1996).

Most supervised learning algorithms minimize a loss that is expressed as a weighted $L_2$ norm. Thus, they don't explicitly provide guarantees in $L_\infty$ norm. R. Munos provided **[error bounds for approximate value iteration](https://www.aaai.org/Papers/AAAI/2005/AAAI05-159.pdf)** in the general case of weighted $L_p$ norms. Those bounds are similar to the one in $L_\infty$ norm, thus justifying the use of supervised learning techniques (such as neural networks or random forests for instance) in Approximate Value Iteration.

**Approximate value iteration generalizes value iteration**

Let us stress this out: approximate value iteration fully generalizes value iteration.  
The Bellman optimality equation defines $Q^*$ as the solution to $Q=T^* Q$. This solution belongs to the space $\mathbb{R}^{SA}$ of functions from $S\times A$ to $\mathbb{R}$. This space of functions is a vector space whose dimension is the cardinality of $S\times A$. Hence, beyond the case of finite state and action spaces, no finite basis of functions can span $\mathbb{R}^{SA}$ and searching for a solution to $Q=T^* Q$ is searching inside an infinite dimensional space.  

When $\mathcal{A}$ turns out to be exact (that is, its approximation error $\epsilon$ is null), then AVI boils down to VI.  
This can happen when $S$ and $A$ are discrete and the functions $\mathbb{R}^{SA}$ are represented exactly as vectors in $\mathbb{R}^{|S||A|}$. But this can happen also in continuous state or action spaces with very specific hypotheses, where the optimality equation admits a closed-form solution. An example of this where the value function is a polynomial function of a continuous state variable is developped in these [two](https://papers.nips.cc/paper_files/paper/2000/hash/09b15d48a1514d8209b192a8b8f34e48-Abstract.html) [papers](https://ieeexplore.ieee.org/abstract/document/5364653).

On the other hand, when the state-action space is continuous, or just too large to be enumerated, AVI trades exact construction of the $Q_{+1} = T^*Q_n$ sequence for scalability, by constructing an approximate sequence $Q_{n+1} = \mathcal{A} T^* Q_n$ that still provides good policies if $\mathcal{A}$ is a good approximation procedure.

**And approximate (modified) policy iteration?**

Approximate Policy Iteration consists in solving the evaluation equation up to a certain precision $\epsilon$ and then taking a greedy improvement step.

We write $V_n$ the approximation of $V^{\pi_n}$ at the end of an evaluation phase in Policy Iteration and suppose that the approximation error is uniformly bounded:
$$\| V^{\pi_n} - V_n \|_\infty \leq \epsilon.$$

Then it is known that, even though the sequence of greedy policies $\pi_n$ does not converge, it oscillates among a set of policies such that:
$$\|V^*-V^{\pi_n}\|_\infty \leq \frac{2\gamma\epsilon}{(1-\gamma)^2}.$$

This result is proven in the **[Neuro-dynamic programming](http://athenasc.com/ndpbook.html)** book by D. P. Bertsekas and J. Tsitsiklis (1996).

So the neighborhood reached has the same size as that of Approximate Value Iteration.

Similar bounds **[error bounds for approximate policy iteration](https://www.aaai.org/Papers/ICML/2003/ICML03-074.pdf)** in weighted $L_p$ norms were provided by R. Munos (2003).  
These bounds were later generalized by the study of **[approximate modified policy iteration](https://icml.cc/2012/papers/608.pdf)** by B. Scherrer et al. (2012).

<div class="alert alert-success">

From now on, we will cast our algorithms as approximate dynamic programming ones, operating on a given function space for Q-functions, part of $\mathbb{R}^{SA}$, regardless of the nature of $S$ and $A$.
</div>

The key idea we develop in this chapter is that one can actually *learn* the sequence of AVI functions using interaction samples rather than *calculate* it using a model.

<center><img src="img/brain.png" width="400px"></img></center>

Although we have introduced a fair amount of abstract concepts, it is important to keep in mind that these maths simply formalize an intuitive cognitive process. By experiencing rewards and punishments, we (humans) incrementally learn to evaluate the outcomes of our actions and then decide to act accordingly. This cognitive process is thus in line with the formalism we have introduced.

# Approximate dynamic programming as a sequence of supervised learning problems

**Prerequisite: random variables and expectations**

You might remember how we defined the *return* random variable $G^\pi(s)$ for a policy $\pi$, in each state $s$, as the $\gamma$-discounted sum of rewards:
$$G^\pi(s) = \sum\limits_{t = 0}^\infty \gamma^t R_t \quad \Bigg| \quad \begin{array}{l}S_0 = s,\\ A_t \sim \pi_t,\\ S_{t+1}\sim p(\cdot|S_t,A_t),\\R_t = r(S_t,A_t,S_{t+1}).\end{array}$$

Let us take inspiration from the previous definition and define a *bootstrapped return* $G^\pi_m(s)$, for $m\geq 1$, as the random variable:
$$G^\pi_m(s) = \sum\limits_{t = 0}^{m-1} \gamma^t R_t + \gamma^m Q(S_m, A_m) \quad \Bigg| \quad \begin{array}{l}S_0 = s,\\ A_t \sim \pi_t,\\ S_{t+1}\sim p(\cdot|S_t,A_t),\\R_t = r(S_t,A_t,S_{t+1}).\end{array}$$

For $m\rightarrow\infty$, $G^\pi_m(s)\rightarrow G^\pi(s)$.

In this specific section, we shall use a generic $T$ operator on Q-functions, which stands for both $T^\pi$ and $T^*$. When necessary, we will replace $T$ by either $T^\pi$ or $T^*$, or by their expressions.

Let us define the dynamic programming sequence of functions $Q_{n+1} = T^* Q_n$.  

<div class="alert alert-warning">

**Exercise:**  
Suppose Q-functions belong to some parametrized set of functions $Q(s,a;\theta)$. 
The parameter of $Q_n$ is noted $\theta_n$. 
We want to approximate $T^*Q_n$ with $Q(s,a;\theta_{n+1})$. 
Write an objective function $L_n(\theta)$ such that $\theta_{n+1} = \arg\min_{\theta} L_n(\theta)$.
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>

Recall that
$$(T^*Q)(s,a) = \mathbb{E}_{s' \sim p(\cdot|s,a)} \left[r(s,a,s') + \gamma \max_{a'} Q(s',a')\right].$$

Let's write $G_n(s,a)$ the random variable in the expectation above. $G_n(s,a)$ is the sum of what we get by applying $a$ in $s$, which is r(s,a,s'), with $s'$ drawn according to $p(\cdot|s,a)$), and the biggest value estimate from $s'$ according to $\theta_n$, discounted by $\gamma$.

For the sake of notation simplicity, let's write, for a given $\theta_n$: 
$$y_n(s,a) = \mathbb{E} \left[ G_n(s,a) \right] = \mathbb{E}_{s' \sim p(\cdot|s,a)} \left[r(s,a,s') + \gamma \max_{a'} Q(s',a';\theta_{n})\right].$$

Then finding $\theta_{n+1}$ given $\theta_n$ is the regression problem that minimizes the objective:
$$L_n(\theta) = \left\| y_n(s,a) - Q(s,a;\theta) \right\|.$$

Note that this objective function depends on the chosen norm. If this function can be optimized and goes to zero, then we have found the true $Q_{n+1}$. If not, then we have found an approximation of it in the norm used to define $L_n$.
</details>

<div class="alert alert-warning">

**Exercise:**  
Use the quadratic loss function to define the previous optimization problem as a risk minimization one. Then write the gradient of this risk with respect to the Q-function's parameters. Use this to introduce a stochastic gradient descent method to find $\theta_{n+1}$.  
Use the (weighted) L2 norm in the loss of the regression problem above, then write the gradient of the loss with respect to the regressor's parameters. Use this to introduce a stochastic gradient descent method to find $\theta_{n+1}$.
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>

The regression problem we consider takes inputs in $S\times A$ and outputs in $\mathbb{R}$. 
To define the risk, we need to introduce the distribution over inputs and outputs. Let us write a distribution $\rho$ over inputs in $S\times A$. For a given $s,a$, the law of $G_n(s,a)$ is fully determined by the transition model $p(s'|s,a)$. 

Our objective function then becomes:
\begin{gather}
L_n(\theta) = \frac{1}{2} \mathbb{E}_{(s,a) \sim \rho(\cdot)}\left[ \big( G_n(s,a) - Q(s,a;\theta) \big)^2 \right],\\
\textrm{with } G_n(s,a) = r(s,a,s') + \gamma \max_{a'} Q(s',a';\theta_{n}),\textrm{ with } s' \sim p(\cdot|s,a).
\end{gather}

In the expression above, $\rho$ is a distribution over the state-action space. Often, it is considered to be the behavior distribution, that is the distribution of samples under the current behavior policy, like $\epsilon$-greedy. Note that this choice is debatable.

So the gradient of this loss is:
\begin{gather}
\nabla_\theta L_n(\theta) = \mathbb{E}_{(s,a) \sim \rho(\cdot)}\left[ \big( y_n(s,a) - Q(s,a;\theta) \big) \nabla_\theta Q(s,a;\theta) \right]\\
\textrm{with } y_n(s,a) = \mathbb{E}_{s' \sim p(\cdot|s,a)} \left[r(s,a,s') + \gamma \max_{a'} Q(s',a';\theta_{n})\right].
\end{gather}

And when we wrap all this together:
$$\nabla_\theta L_n(\theta) = \mathbb{E}_{(s,a) \sim \rho(\cdot)}\left[ \left( \mathbb{E}_{s' \sim p(\cdot|s,a)} \left[r(s,a,s') + \gamma \max_{a'} Q(s',a';\theta_{n})\right] - Q(s,a;\theta) \right) \nabla_\theta Q(s,a;\theta) \right]$$

$$\nabla_\theta L_n(\theta) = \mathbb{E}_{\substack{(s,a) \sim \rho(\cdot)\\ s' \sim p(\cdot|s,a)}}\left[ \left( r(s,a,s') + \gamma \max_{a'} Q(s',a';\theta_{n}) - Q(s,a;\theta) \right) \nabla_\theta Q(s,a;\theta) \right]$$

We can build a Monte Carlo estimate of this gradient, given a mini-batch of independently and identically drawn samples $\left\{\left(s_i,a_i,r_i,s'_i\right)\right\}_{i\in [1,B]}$, with $(s,a) \sim \rho(\cdot)$ and $s' \sim p(\cdot | s,a)$:
$$\nabla_\theta L_n(\theta) \approx d_n(\theta) = \sum_{i=1}^B \left[ \left( r_i + \gamma \max_{a'} Q(s_i',a';\theta_{n}) - Q(s_i,a_i;\theta) \right) \nabla_\theta Q(s_i,a_i;\theta) \right].$$

The stochastic gradient descent procedure builds a sequence of parameter values $\theta_k$ such that:
$$\theta_{k+1} \leftarrow \theta_{k} - \alpha d_n(\theta_{k})$$

By repeating such gradient steps, one progressively minimizes $L_n(\theta)$ and finds $\theta_{n+1}$.
</details>

# Policy evaluation as Stochastic Approximation: Temporal Differences

Recall that evaluating $Q^\pi(s,a)$ is estimating the mathematical expectation of $G^\pi(s,a)$.

Stochastic approximation theory tells us that, for a given $s,a$ pair, given a series $g^\pi_t$ of independent realizations of $G^\pi(s,a)$, the sequence
$q_{t+1} = q_t + \alpha_t \left(g^\pi_t - q_t\right)$
converges to $\mathbb{E}\left(G^\pi(s,a)\right)$, if the sequence of $\alpha_t$ respects the Robbins-Monro conditions ($\sum_t \alpha_t = \infty$ and $\sum_t \alpha_t^2 < \infty$).

**An intuitive reminder on Stochastic Approximation.**  

    
For those unfamiliar with stochastic approximation procedures, we can understand the previous update as: $g^\pi_t$ are sample estimates of $\mathbb{E}\left(G^\pi(s,a)\right)$. If I already have an estimate $q_t$ of $\mathbb{E}\left(G^\pi(s,a)\right)$ and I receive a new sample $g^\pi_t$, I should "pull" my previous estimate towards $g^\pi_t$. But $g^\pi_t$ carries a part of noise, so I should be cautious and only take a small step $\alpha$ in the direction of $g^\pi_t$.
    
In turn, the convergence conditions simply state that any value $Q^\pi(s,a)$ should be reachable given any initial guess $Q(s,a)$, no matter how far is this first guess from $Q^\pi(s,a)$; hence the $\sum\limits_{t=0}^\infty \alpha_t = \infty$. However, we still need the step-size to be decreasing so that we don't start oscillating around $Q^\pi(s,a)$ when we get closer; so to insure convergence we impose $\sum\limits_{t=0}^\infty \alpha_t^2 < \infty$.

So this provides us with a way to estimate $Q^\pi(s,a)$ from experience samples rather than from a model.  
<div class="alert alert-success">
    
**Policy evaluation as stochastic approximation**  
If we can obtain independent realizations $g^\pi(s,a)$ of $G^\pi(s,a)$ in all $s,a$, we can perform stochastic approximation updates of $Q$ under the form:
$$Q(s,a) \leftarrow Q(s,a) + \alpha \left(g^\pi(s,a) - Q(s,a)\right).$$
Then $Q$ converges to $Q^\pi$.
</div>

A more modern formulation of Stochastic Approximation is Stochastic Gradient Descent. So we will slightly generalize the formulation above.  

$Q^\pi$ is the function that minimizes
$$L(Q) = \frac{1}{2} \int_{S\times A} \left[ Q(s,a) - \mathbb{E}\left(G^\pi(s,a)\right)\right]^2 dsda.$$

Recall that in the most general case, $Q$ is a function, but for the sake of clarity, we will momentarily suppose that $S\times A$ is finite and thus $Q$ is equivalent to the vector of all $Q(s,a)$ values.

Then, minimizing $L(Q)$ can be done via gradient descent:
$$\nabla_Q L(Q) = \int_{S\times A} \left[ Q(s,a) - \mathbb{E}\left(G^\pi(s,a)\right) \right] \nabla_Q Q(s,a) dsda.$$

Suppose we have a set of independently drawn states and actions $\left\{(s_i, a_i)\right\}_{i\in [1,N]}$. Then, this gradient can be approached via a Monte Carlo estimator:
$$\sum_{i=1}^N \left[ Q(s_i,a_i) - \mathbb{E}\left(G^\pi(s_i,a_i)\right)\right] \nabla_Q Q(s_i, a_i).$$

In our example where $Q$ is the vector of values taken in each state and action, 
$\nabla_Q Q(s_i,a_i) = \left[ \begin{array}{c} 0\\ \vdots\\ 0\\ 1 \\ 0\\ \vdots\\ 0 \end{array} \right]$ 
where the "1" is at the position corresponding to $s_i,a_i$ in the vector $Q$.

As for Stochastic Approximation, if we can obtain independent realizations $g^\pi(s_i,a_i)$ of $G^\pi(s_i,a_i)$, then we can estimate this gradient as:
$$d = \sum_{i=1}^N \left[ Q(s_i,a_i) - g^\pi(s_i,a_i)\right] \nabla_Q Q(s_i,a_i).$$

And thus we have the Stochastic Gradient Descent update:
$$Q \leftarrow Q - \alpha \sum_{i=1}^N \left[ Q(s_i,a_i) - g^\pi(s_i,a_i)\right] \nabla_Q Q(s_i,a_i)$$

This update mechanism yields a sequence $Q_t$ of value functions that converges to $Q^\pi$ if the gradient steps $\alpha$ respect Robbins-Monro conditions.

<div class="alert alert-success">
    
**Policy evaluation as Stochastic Gradient Descent**  
If we can obtain independent realizations $g^\pi(s,a)$ of $G^\pi(s,a)$ in all $s,a$, we can perform Stochastic Gradient Descent updates on $Q$:
$$Q \leftarrow Q + \alpha \sum_{i=1}^N \left[ g^\pi(s_i,a_i) - Q(s_i,a_i)\right] \nabla_Q Q(s_i,a_i).$$
Then $Q$ converges to $Q^\pi$.
</div>

Note that if $N=1$, the update above falls back to the Stochastic Approximation update: having a sample in $s_i,a_i$ only updates $Q(s_i,a_i)$.

For the sake of simplicity, we will keep the Stochastic Approximation perspective in further developments. The transition to SGD is straightforward.

So, overall, if we manage to draw independent samples $g^\pi(s_i,a_i)$ of $G^\pi(s,a)$ in all $s,a\in S\times A$, we can **learn** the value $Q^\pi$ (or $V^\pi$) of policy $\pi$.  

Consider the sample $(s_t,a_t,r_t,s_{t+1})$ obtained at time $t$.

Once this transition is over we can update our knowledge of $Q(s_t, a_t)$ by using $r_t+\gamma Q(s_{t+1},\pi(s_{t+1}))$. This estimate uses $Q(s_{t+1},\pi(s_{t+1}))$ to *bootstrap* [*] the estimator of $Q(s_t, a_t)$.

[*] This *bootstrap* operation has nothing to do with the statistical procedure of *[bootstrapping](https://en.wikipedia.org/wiki/Bootstrapping_(statistics))*.

This idea, which was first introduced in R. Sutton's **[Learning to predict by the methods of temporal differences](https://link.springer.com/article/10.1007/BF00115009)** article, has a strong parallel with the evaluation equation. This equation was:
$$Q^\pi \left(s,a\right) = \mathbb{E}_{s' \sim p\left(s'|s,a\right)} \left[ r\left(s,a,s'\right) + \gamma Q^\pi \left(s', \pi(s')\right) \right]$$  

The key remark is that the sample $g^\pi_t$ of $Q^\pi(s_t,a_t)$ can be built by summing $r_t$ and $\gamma Q_t(s_{t+1}, \pi(s_{t+1}) )$:
$$g_t = r_t + \gamma Q_t(s_{t+1}, \pi(s_{t+1})).$$

Note that in the expression above, we have used $Q_t$ to emphasize that we use the function $Q$ as it was at time step $t$, to define the target $g^\pi_t$ used in the update that will provide $Q_{t+1}$.

Formally, this comes directly from the evaluation operator. Let's rewrite $T^\pi$ in terms of random variables.
$$(T^\pi Q)(s,a) = \mathbb{E}_{R,S'}\left[ R + \gamma Q(S', \pi(S')) \right]$$

Since $Q^\pi$ is the fixed point of $T^\pi$, by taking $g_t = r_t + \gamma Q_t(s_{t+1},\pi(s_{t+1}))$ we are taking one stochastic approximation step in the direction of $T^\pi Q_t$. 

**Bootstrapping** (in this particular context) is the operation of using the value of $Q_t(s_{t+1},\pi(s_{t+1}))$ in the update of $Q$.

Then the stochastic approximation update becomes what is called the **TD(0) update**:
<div class="alert alert-success">
    
**TD(0) update:**  
$$Q(s_t,a_t) \leftarrow Q(s_t,a_t) + \alpha \left(r_t + \gamma Q(s_{t+1}, \pi(s_{t+1})) - Q(s_t,a_t)\right).$$
    
This update consists in taking one stochastic approximation step in the direction of $T^\pi Q$.
</div>

The SGD update is almost the same.

Let's insist on this point:  
TD(0) does not directly solve $Q=\mathbb{E}\left[\sum_t\gamma^t R_t \right]$ (this is what other methods, called *Monte Carlo*, do). Instead, it implements stochastic approximation on top of the repeated application of the $T^\pi$ operator. So it solves $Q_{n+1} = T^\pi Q_n$. At each step $t$, it takes the current value function $Q_t$, draws one or several samples from $T^\pi Q_t$ and approximates $T^\pi Q_t$ by taking one step of gradient descent from $Q_t$.

$\delta_t=r_t + \gamma Q_t(s_{t+1},\pi(s_{t+1})) - Q_t(s_t,a_t)$ is called the prediction **temporal difference** (hence the name of the algorithm - the "0" won't be explained here). It is the difference between our estimate $Q_t(s_t,a_t)$ *before* obtaining the information of $r_t$, and the bootstrapped value $r_t + \gamma Q_t(s_{t+1},\pi(s_{t+1}))$.
<div class="alert alert-success"><b>Temporal difference:</b>
$$\delta=r + \gamma Q(s',\pi(s')) - Q(s,a)$$
</div>

Now it seems obvious that if some state-action pair $s,a$ is never visited, then no update of its $Q(s,a)$ can ever take place. Therefore, for the TD(0) update to converge, we need to guarantee that all state action pairs will be visited frequently enough for $Q$ to converge to $Q^\pi$.

<div class="alert alert-success"><b>TD(0) temporal difference update on $Q$-functions:</b><br>
For a sample $(s,a,r,s')$, the temporal difference is:
$$\delta = r + \gamma Q(s',\pi(s')) - Q(s,a)$$
And the TD update is:
$$Q(s,a) \leftarrow Q(s,a) + \alpha \left[ r + \gamma Q(s',\pi(s')) - Q(s,a) \right]$$
As long as all state-action pairs $(s,a)$ are sampled infinitely often as $t\rightarrow\infty$, and under the Robbins-Monro conditions, this procedure converges to $Q^\pi$.
</div>

Interestingly, this algorithm puts restrictions on the policy we apply when interacting with the environment. We will call such a policy a **behavior policy**. The behavior policy and the policy being learned might be different (in the case of TD(0), this even is an obligation since we need to enforce visitation of all state-action pairs).

Vocabulary: **Off-policy** evaluation algorithms can use a behavior policy that is different than the policy being evaluated.

<div class="alert alert-warning"><b>Live coding:</b><br>
Let's implement TD(0) on $Q$-functions.<br>
To insure that all states and actions are sampled infinitely often, we take a behavior policy that acts randomly in each state.<br>
We take $\gamma=0.9$ and run the algorithm for 2000000 time steps.<br>
To keep things simple, we take a constant $\alpha=0.01$.
</div>

In [ ]:
fl.RIGHT

In [ ]:
gamma = 0.9
alpha = 0.01
max_steps=1000000
Qtd = np.zeros((env.observation_space.n, env.action_space.n))
Qtrue = Qpi_sequence[-1]

error = np.zeros((max_steps))
x,_ = env.reset()
for t in range(max_steps):
    a = np.random.randint(4)
    y,r,d,_,_ = env.step(a)
    Qtd[x][a] = Qtd[x][a] + alpha * (r+gamma*Qtd[y][fl.RIGHT]-Qtd[x][a])
    error[t] = np.max(np.abs(Qtd-Qtrue))
    if d==True:
        x,_ = env.reset()
    else:
        x=y

In [ ]:
print("Max error:", np.max(np.abs(Q-Qtrue)))
plt.figure()
plt.plot(error)
plt.figure()
plt.semilogy(error);

Many improvements, like [Experience Replay (Lin, 1992)](https://link.springer.com/article/10.1007/BF00992699) for instance, produce better gradient estimates. We won't cover them here and reserve them for future classes.

# Approximate Value Iteration as Stochastic Approximation: Q-learning

We can directly adapt the idea of temporal difference learning to the Approximate Value Iteration case.

In this case, we want to learn $T^* Q_t$ (instead of $T^\pi Q_t$) so our samples are:
$$g_t = r_t + \gamma \max_{a'} Q_t(s_{t+1},a').$$

And the learning algorithm becomes the famous Q-learning algorithm, introduced by C. J. Watkins in his [PhD thesis](http://www.cs.rhul.ac.uk/~chrisw/new_thesis.pdf) in 1989:
<div class="alert alert-success"><b>Q-learning</b><br>
For a sample $(s,a,r,s')$, the temporal difference is:
$$\delta = r + \gamma \max_{a'} Q(s',a') - Q(s,a)$$
And the TD update is:
$$Q(s,a) \leftarrow Q(s,a) + \alpha \left[ r + \gamma \max_{a'} Q(s',a') - Q(s,a) \right]$$
As long as all state-action pairs $(s,a)$ are sampled infinitely often as $t\rightarrow\infty$, and under the Robbins-Monro conditions, this procedure converges to $Q^*$.
</div>

To implement a Q-learning algorithm, one needs to decide on a behavior policy. As for TD(0), Q-learning will converge to $Q^*$, provided that all states and actions are visited infinitely often. It is actually notable that $Q$ converges to $Q^*$ even if the behavior policy does not. But it also looks like a waste of computational resources to keep exploring uniformly around the starting state.

This tradeoff between exploring new actions and exploiting what has already been inferred in $Q$ is called the **exploration versus exploitation tradeoff**. It is a crucial problem that strongly affects the ability of the algorithm to discover new, interesting rewards.

Here we will implement a rather naive tradeoff strategy called an $\epsilon$-greedy behavior. It consists in picking the $Q$-greedy action with probability $1-\epsilon$ and a random action with probability $\epsilon$.

$\epsilon$ will start at 1 and then will periodically be divided by 2.

<div class="alert alert-warning">
    
**Live coding:**

Write a function that picks an epsilon-greedy action.
</div>

In [ ]:
def epsilon_greedy(Q, s, epsilon):
    a = np.argmax(Q[s,:])
    if(np.random.rand()<=epsilon): # random action
        aa = np.random.randint(env.action_space.n-1)
        if aa==a:
            a=env.action_space.n-1
        else:
            a=aa
    return a

<div class="alert alert-warning">
    
**Live coding:**

Write a Q-learning algorithm on FrozenLake. Keep track of the error w.r.t. $Q^*$ and the number of times each state-action pair is visited.
</div>

In [ ]:
# Let's restart from the previous Qpi
Qql = Qpi_sequence[-1]
max_steps = 2000000

# Q-learning
count = np.zeros((env.observation_space.n,env.action_space.n)) # to track update frequencies
epsilon = 1
x,_ = env.reset()
for t in range(max_steps):
    if((t+1)%1000000==0):
        epsilon = epsilon/2
    a = epsilon_greedy(Qql,x,epsilon)
    y,r,d,_,_ = env.step(a)
    Qql[x][a] = Qql[x][a] + alpha * (r+gamma*np.max(Qql[y][:])-Qql[x][a])
    count[x][a] += 1
    if d==True:
        x,_ = env.reset()
    else:
        x=y

# Q-learning's final value function and policy
print("Max error:", np.max(np.abs(Qql-Qopt_sequence[-1])))

<div class="alert alert-warning">
    
**Live coding:**

Display the visitation frequency</div>

In [ ]:
count_map = np.zeros((env.unwrapped.nrow, env.unwrapped.ncol, env.action_space.n))
for a in range(env.action_space.n):
    for x in range(env.observation_space.n):
        row,col = to_row_col(x)
        count_map[row, col, a] = count[x,a]

fig, axs = plt.subplots(ncols=4)
for a in range(env.action_space.n):
    name = "a = " + actions[a]
    axs[a].set_title(name)
    axs[a].imshow(np.log(count_map[:,:,a]+1), interpolation='nearest')
    #print("a=", a, ":", sep='')
    #print(count_map[:,:,a])
plt.show()
print(env.render())

<div class="alert alert-warning">
    
**Live coding:**

Display the final policy.</div>

In [ ]:
def greedyQpolicy(Q):
    pi = np.zeros((env.observation_space.n),dtype=np.int32)
    for s in range(env.observation_space.n):
        pi[s] = np.argmax(Q[s,:])
    return pi

def print_policy(pi):
    for row in range(env.unwrapped.nrow):
        for col in range(env.unwrapped.ncol):
            print(actions[pi[to_s(row,col)]], end='')
        print()
    return

print("Final epsilon:", epsilon)
pi_ql = greedyQpolicy(Qql)
print("Greedy Q-learning policy:")
print_policy(pi_ql)

# Summary

Previous chapters have shown how to charaterize and find optimal policies given the MDP model. We have built on the results of Approximate Dynamic Programming to **learn** optimal value functions from interaction samples.

So we have built the third stage of our three-stage rocket defined in the introduction chapter.

<div class="alert alert-success">

**How do we learn an optimal strategy?**  
To learn an optimal strategy, we rely on a stochastic approximation of $Q^*$, given samples drawn from the MDP. This stochastic approximation of $Q^*$ is actually a stochastic approximation of the $Q_n$ sequence of approximate value iteration. In the end, we need to find good approximation architectures and to control the exploration versus exploitation tradeoff.
</div>

Let's take a step back and provide a more general summary this far.

<div class="alert alert-success">

**What is Reinforcement Learning?**  
RL is the discipline that studies the *learning* process of optimal control policies in the MDP framework.  
Its roots overlap Cognitive Psychology, Control Theory, Artificial Intelligence, Machine Learning.
</div>

<div class="alert alert-success">

**What are the building blocks of RL?**  
RL is built upon the framework of Markov Decision Processes.  
It draws from the characterization of optimal policies that maximize a given criterion, notably through Bellman's equations.  
It learns (notably through stochastic approximation or SGD) solutions to these equations using interaction samples.
</div>

Of course we have barely touched the surface of RL for now. We haven't explored the weaker notion of optimality and the direct policy search methods sketched out in chapter 1 for instance. The overall goal of these first chapters was to acquire a common vocabulary and set of concepts, so that you become comfortable with the objects often manipulated in RL. 

## Three intrinsic challenges in Reinforcement Learning

From here, we can identify three challenges which make the RL problem intrinsically difficult:
<div class="alert alert-success">

**Intrinsic challenges in RL:**  
- function approximation,
- the improvement problem,
- the exploration versus exploitation trade-off.
</div>

These three challenges are quite independent and we could study them in any order.

As we have seen, **function approximation** is key in finding good policies. Although it needs not be tackled with stochastic gradient descent, the interplay between Deep Learning and Reinforcement Learning has triggered major advances in RL. 
Chapter 4 is dedicated to this topic and will lead us to manipulate function approximators in AVI, including deep neural networks. 

The ideas we developped in this class relied on estimating value functions to deduce greedy policies. Finding such greedy policies was made easy because actions were discrete. But **finding improving policies** is actually a challenge in itself. This problem is present both when one searches for a greedy action with respect to $Q$, and when one directly aims at solving the $\max_\pi J(\pi)$ problem without going through the proxy of the optimality equation. Chapter 5 will take us towards the realm of continuous actions in AVI. Chapter 6 will tackle direct policy search and resolution of the $\max_\pi J(\pi)$ problem, notably through policy gradient algorithms.

Behavior policies are a cornerstone of RL: which action should one take to obtain informative samples? Should one explore uniformly the environment or rather follow a policy that takes the system towards promising states before exploring more agressively? This is called the **tradeoff between exploration and exploitation**. The last two sessions of the class will be dedicated to properly studying this tradeoff through stochastic bandits, which leads to the UCT and Monte Carlo Tree Search algorithms that are at the root of [alphaGo](https://www.youtube.com/watch?v=WXuK6gekU1Y).  

## Subtopics in RL

Beyond these three intrinsic challenges, there are countless, context-dependent, open questions in RL, that form a span of specific questions:
- Hierarchical RL
- RL from human feedback
- World (surrogate) models
- Multi-agent RL
- Partially observable MDPs
- Robust RL
- Offline RL
- Consolidation and Transfer in RL
- Causal RL
- and many more (not counting all the application fields)!

# Homework

## Homework: variations on TD(0)

<div class="alert alert-warning">

**Exercise**
Write a function `TD_Qeval` that runs TD(0) on tabular $Q$ functions for a discrete state-action environment and a given policy (it's almost the same code than in section 7.1). Add an option for providing the true $Q$ function and monitoring the error along training. Use the code below to compute the $Q$ function of the policy that always goes right in FrozenLake and plot the evolution of the error between `Qtrue` (model-based computations of previous sections) and $Q$.
</div>

In [ ]:
# %load solutions/RL3_exercise5.py
### WRITE YOUR CODE HERE
# If you get stuck, uncomment the line above to load a correction in this cell (then you can execute this code).

In [ ]:
import gym
import gym.envs.toy_text.frozen_lake as fl
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

env = gym.make('FrozenLake-v0')
pi0 = fl.RIGHT*np.ones((env.observation_space.n),dtype=np.int)
gamma = 0.9
alpha = 0.001
Qpi0 = np.zeros((env.observation_space.n, env.action_space.n))

Qpi0, error = TD_Qeval(pi=pi0, max_steps=3000000, env=env, alpha=alpha, gamma=gamma, Qpi=Qpi0, Qtrue=Qpi_sequence[-1])
print("Max error:", np.max(np.abs(Qpi0-Qpi0true)))
plt.plot(error)
plt.figure()
plt.semilogy(error);

<div class="alert alert-warning">

**Exercise**

We wrote the derivation of (batch, SGD) TD(0) using $Q$ functions defined on the discrete $S\times A$ space, and taking the gradient with respect to the vector $Q$. Write the same derivation with a parametric state value function $V_\theta$, then with a parametric state-action value function $Q_\theta$.
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>

We have $V^\pi(s) = \mathbb{E} (G^\pi(s))$. So we consider samples $g^\pi(s)$ of $G^\pi(s)$.

The loss function becomes $L(\theta) = \int_S \left[ V_\theta(s) - \mathbb{E}\left(G^\pi(s)\right)\right]^2 ds$.
    
The gradient estimator becomes:
$$d = \sum_{i=1}^N \left[ V_\theta(s_i) - g^\pi(s_i)\right] \nabla_\theta V_\theta(s_i).$$

And so the update becomes:
$$\theta \leftarrow \theta + \alpha \sum_{i=1}^N \left[ g^\pi(s_i) - V_\theta(s_i)\right] \nabla_\theta V_\theta(s_i)$$
    
For a parametric $Q_\theta$:
$$\theta \leftarrow \theta + \alpha \sum_{i=1}^N \left[ g^\pi(s_i,a_i) - Q_\theta(s_i,a_i)\right] \nabla_\theta Q_\theta(s_i).$$

</details>

<div class="alert alert-warning">
    
**Exercise:**  
We derived the TD(0) algorithm for $Q$ functions. Let us draw inspiration from the previous exercise and the derivation of this section to write TD(0) on $V$ functions.
- First, recall $T^\pi$ in terms of an expectation over random variables $R$ and $S'$.
- Then define a bootstrapped sample $g_t$ of this expectation.
- Finally write the TD(0) SGD update on parametric state value functions, and the corresponding temporal difference.
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>

- $T^\pi$ operator: $(T^\pi V)(s) = \mathbb{E}_{R,S'}\left[ R + \gamma V(S') \right]$
- Bootstrap sample: $g_t = r_t + \gamma V_t(s_{t+1})$.
- The TD(0) update is $\theta \leftarrow \theta + \alpha \sum_{i=1}^N \left[ r_i + \gamma V_\theta(s_{i+1}) - V_\theta(s_i) \right] \nabla_\theta V_\theta(s_i)$.  
    The temporal difference is $\delta = r_t + \gamma V(s_{t+1}) - V(s_t)$.

</details>

<div class="alert alert-warning">
    
**Exercise:**  
When deriving TD(0) on $Q$ functions, we wrote that we needed to enforce the visitation of every state-action pair when obtaining samples from the MDP. Is it still the case for TD(0) on $V$ functions? Is TD(0) on $V$ functions an off-policy algorithm?
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>

We need $g_t$ to be a sample of $G^\pi(s_t) = R_t + \gamma V(S_{t+1})$. So,  $r_t$ should be a sample of $R_t$, that is the reward obtained by taking action $\pi(s_t)$ in $s_t$. Additionally, $S_{t+1}$ should be drawn according to $p(\cdot | s_t,\pi(s_t)$. So the behavior policy **needs** to be $\pi$, otherwise the samples lose all meaning. TD(0) on $V$ functions is an **on-policy** algorithm: it's behavior policy is constrained to be the one under evaluation.

One could remark that since the policy applied is $\pi$, we cannot guarantee that all states will be visited. However, the visited states will be those reachable by $\pi$, from the initial state. Refer to section 5's exercise on the stationary distribution for a discussion on the reachable states and their visitation frequency.

</details>

<div class="alert alert-warning">
    
**Exercise:**  
Consider the policy that always moves right in FrozenLake and implement a TD(0) estimation for $V^\pi$.  
We can tolerate a 0.001 error on $V^\pi$ so, to keep things simple, we set $\alpha=0.001$.  
Take $\gamma = 0.9$ and run the algorithm for 2000000 time steps.  
After each episode, compare the value function obtained with that computed with the model-based approaches of section 6.
</div>

In [ ]:
# %load solutions/RL3_exercise2.py
### WRITE YOUR CODE HERE
# If you get stuck, uncomment the line above to load a correction in this cell (then you can execute this code).

In general, we will call $\beta$ the *behavior policy*. It is the policy being applied to interact with the environment. Off-policy evaluation algorithms can use a behavior policy that is different than the policy being evaluated. We will call *behavior distribution* the distributions $\rho^\beta(s)$ over states and $\rho^\beta(s,a)$ over state-action pairs, induced by applying $\beta$.

## Homework: delayed updates and experience replay for TD(0)

We have seen that TD(0), at each time step, takes a gradient step in the direction of $T^\pi Q$.

The result of this gradient step is an approximation of $T^\pi Q$.

We saw in section 6 that despite an approximation operator $\mathcal{A}$ with controlled error, the sequence $Q_{n+1} = \mathcal{A} T^* Q_n$ still converged to a neighborhood of $Q^*$. The same result holds for the sequence $Q_{n+1} = \mathcal{A} T^\pi Q_n$.

One single step of stochastic gradient descent makes for a poor approximator. Given a fixed function $Q_n$, if we repeat a certain number $C$ of such gradient steps, we can hope to obtain a better estimate of $T^\pi Q_n$. So there is an interest in keeping two $Q$ functions. The first is the current estimator, which plays the role of $Q_n$, upon which we apply $T^\pi$, and which we call the *target* function $Q^-$. The second is the one we actually optimize upon and which aims at approximating $T^\pi Q^-$; we write it $Q$. Every $C$ gradient steps, we replace $Q^-$ by $Q$ and repeat.

Consequently, this procedure of **delayed updates** trades off advancing in the $Q_{n+1} = \mathcal{A} T^\pi Q_n$ sequence for better approximation properties for $\mathcal{A}$.

This makes more apparent the remark made before that TD(0) actually solves the $Q_{n+1} = T^\pi Q_n$ sequence and thus successively minimizes a sequence of losses:
$$L_n(Q) = \| Q - T^\pi Q_n \|_2.$$
The loss changes everytime we replace $Q_n$ by $Q$.

<div class="alert alert-warning">
    
**Exercise:**  
What's the value of $C$ in vanilla TD(0)?
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>

Vanilla TD(0) replaces $Q^-$ by $Q$ at every step, so $C=1$.
</details>

We have written several times the update of TD(0) with a batch of sampels. But in order to be able to perform stochastic gradient steps using more than one sample, one needs to keep samples in memory.

Recall the loss we defined to introduce the stochastic gradient update:
$$L(Q) = \int_{S\times A} \left[ Q(s,a) - \mathbb{E}\left(G^\pi(s,a)\right)\right]^2 ds da.$$

Recall also that $d = \sum_{i=1}^N \left[ Q(s_i,a_i) - g^\pi(s_i,a_i)\right] \nabla_Q Q(s_i,a_i)$ is an unbiased estimate of $\nabla_Q L(Q)$ only if the $g^\pi(s_i,a_i)$ are drawn **independently** and **identically** according to the distribution of $G^\pi(s,a)$.

This last condition can only be verified if 
1. the $s_i,a_i$ are drawn independently of each other and always according to the same distribution $\rho(s,a)$, and
2. given $s_i,a_i$, the realizations $g^\pi(s_i,a_i)$ are drawn independently of each other and according to the distribution of $G^\pi(s_i,a_i)$.

<div class="alert alert-warning">
    
**Exercise:**  
What do you think? Is condition 1 verified in vanilla TD(0)? What about condition 2?
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>

Let's write $(s_i,a_i,r_i,s'_i)$ the $i$th sample and $\beta$ the behavior policy.
    
Condition 1 is not verified for vanilla TD(0). It is true that if the behavior policy is constant, then on average the state samples $s_i$ are drawn **identically**, according this policy's stationary state distribution $\rho^\beta(s)$ and the action samples are drawn according to $\beta(\cdot | s)$. However, successive samples are not drawn **independently** by definition, since $\mathbb{P}(S_{t+1})$ is actually conditioned by $S_t$ and $A_t\sim\beta(\cdot | S_t)$.

    
Condition 2, on the other hand is easier to verify for TD(0) updates: since the reward $r_i$ and the next state $s'_i$ are only conditioned by $s_i$ and $a_i\sim \beta(\cdot | s_i)$, the $g^\pi(s_i,a_i) = r_i + \gamma Q(s'_i,\pi(s'_i))$ are all drawn identically and independently of each other.
</details>

Despite this, TD(0) with tabular function representation still converges as long as states and actions are tried frequently enough. In some cases of function approximation this is also still true. But we might have found a major issue here in the most general case.

One way to (approximately) recover the conditional independence of sampled states $s_i$ is to store a large number of samples $(s_i,a_i,r_i,s'_i)$ in memory, and draw samples uniformly at random from this memory for the TD(0) updates. This idea was first introduced by Lin in his 1992 **[Self-improving reactive agents based on reinforcement learning, planning and teaching](https://link.springer.com/article/10.1007/BF00992699)** article, under the name of *experience replay* (although his derivation was not exactly the same and applied to the Q-learning algorithm which we reserve for another class).

The memory of samples is generally called an *experience replay memory* or *experience replay buffer*, since it allows the learning agent to store past experience in memory and recall it (replay it) as many times as necessary to facilitate learning.

Drawing uniformly randomly from a replay buffer preserves the stationary distribution which generated the samples and breaks the conditional dependency between successive samples in a trajectory.

Combined with the delayed updates introduced earlier, this yields a general, practical algorithm:

<div class="alert alert-success">
    
**TD(0) with delayed updates and experience replay:**  
Given a set of samples $\left\{(s_i,a_i,r_i,s'_i)\right\}_{i\in [1,N]}$ all drawn from a fixed behavior distribution, and a *target* function $Q^-$, the gradient update is:
$$Q \leftarrow Q + \alpha \sum_{i=1}^N \left[ r_i + \gamma Q^-(s'_{i},\pi(s'_i)) - Q_\theta(s_i,a_i) \right] \nabla_Q Q(s_i,a_i)$$
As long as all state-action pairs $(s,a)$ are sampled infinitely often as $t\rightarrow\infty$, under the Robbins-Monro conditions, and under repeated substitution of $Q^-$ by $Q_\theta$ every $C$ gradient updates, this procedure converges to $Q^\pi$ that minimizes $\|Q - Q^\pi \|_2$.
</div>

We can write the same update in the case of parametric function approximation.

<div class="alert alert-success">
    
**TD(0) with delayed updates, experience replay and parametric function approximation:**  
Given a set of samples $\left\{(s_i,a_i,r_i,s'_i)\right\}_{i\in [1,N]}$ all drawn from a fixed behavior distribution, a *target* function $Q^-$,  and a parametric function approximator $Q_\theta$, the gradient update is:
$$\theta \leftarrow \theta + \alpha \sum_{i=1}^N \left[ r_i + \gamma Q^-(s'_{i},\pi(s'_i)) - Q_\theta(s_i,a_i) \right] \nabla_\theta Q_\theta(s_i,a_i)$$
As long as all state-action pairs $(s,a)$ are sampled infinitely often as $t\rightarrow\infty$, under the Robbins-Monro conditions, and under repeated substitution of $Q^-$ by $Q_\theta$ every $C$ gradient updates, this procedure converges to $\theta^\pi$ that minimizes $\|Q_\theta - Q^\pi \|_2$.
</div>

## Homework: the importance of the behavior distribution

We have written above that TD(0) on $Q$ functions is an off-policy algorithm since it will estimate $Q^\pi$ whatever the behavior policy is, as long as this policy explores all state-action pairs infinitely often.

This is actually only true if we are working with tabular representations of $Q$ functions.

Let's return to the gradient descent formulation for a minute. We wrote that the loss was:
$$L(Q) = \int_S \int_A \left[ Q(s,a) - \mathbb{E}\left(G^\pi(s,a)\right)\right]^2 ds da.$$
    
And we wrote the Monte Carlo estimate of this loss' gradient:
$$d = \sum_{i=1}^N \left[ Q(s_i,a_i) - g^\pi(s_i,a_i)\right] \nabla_Q Q(s_i,a_i).$$

This is actually only correct if the samples $(s_i, a_i)$ are drawn uniformly in $S \times A$. But in practice, the samples are drawn in $S\times A$ according to the behavior distribution $\rho^\beta(s,a)$.

Let us get back to the general formulation of stochastic gradient descent. For this, we will introduce a distribution $\Gamma$ over $S\times A\times \mathbb{R}$ that describes the joint distribution of state-action pairs $(s,a)$ and long term returns $G^\pi(s,a)$ under policy $\pi$. So, given a behavior distribution $\rho^\beta(s,a)$, essentially:
$$\Gamma(s,a,g^\pi) = \rho^\beta(s,a) \mathbb{P}(G^\pi(s,a)=g^\pi|s,a)$$

Then finding $Q^\pi$ amounts to solving the minimization problem $\min_Q L(Q)$ with:
\begin{align*}
L(Q) &= \mathbb{E}_{s,a,g^\pi \sim \Gamma} \left[ \left(Q(s,a) - g^\pi\right) \right],\\
     &= \int_S \int_A \int_\mathbb{R} \left[ Q(s,a) - g^\pi \right]^2 \Gamma(s,a,g^\pi) ds da dg^\pi.
\end{align*}

Then, **if the samples $(s,a,g^\pi)$ are drawn according to $\Gamma(s,a,g^\pi)$**, the Monte Carlo estimate
$$d = \sum_{i=1}^N \left[ Q(s_i,a_i) - g^\pi(s_i,a_i)\right] \nabla_Q Q(s_i,a_i)$$
is actually a correct estimate of $\nabla_Q L(Q)$.

What does it mean that $(s,a,g^\pi)$ are drawn according to $\Gamma(s,a,g^\pi)$? It means state-action pairs are drawn according to $\rho^\beta(s,a)$ and the $g^\pi(s_i,a_i)$ follow the distribution of $G^\pi(s,a)$

Otherwise, this Monte Carlo estimate it is a biased estimator that minimizes another loss function; for example one defined by another behavior distribution $\rho^\beta(s,a)$.

So when we are summing elements drawn from the replay memory, or when we are using single samples drawn from the interaction with the MDP, we are actually minimizing the loss function defined specifically by $\rho^\beta(s,a)$. And in the end, for another behavior distribution, the resulting minimizer $Q$ of the loss might be different from the one obtained by using samples collected with $\rho^\beta$.

So, does that mean TD(0) on $Q$ functions is not really off-policy?

In the case of tabular representations, the fact that the gradients $\nabla_Q Q(s_i,a_i)$ are actually indicator functions of $(s_i, a_i)$ limits the impact of having different distributions on $S\times A$. In this case, the minimizers of $L(Q)$ are actually all the same across behavior distributions, as long as the behavior distribution's support spans fully $S\times A$.

However, this nice property is lost in the general case, in particular in the case of function approximation $Q_\theta$. This has motivated the introduction of the **[Gradient Temporal Difference](https://proceedings.neurips.cc/paper/2008/hash/e0c641195b27425bb056ac56f8953d24-Abstract.html)** family of algorithms. This topic is beyond the scope of this class but the interested reader is encouraged to look at **[H. R. Maei's PhD thesis](https://era.library.ualberta.ca/items/fd55edcb-ce47-4f84-84e2-be281d27b16a)**.

## Homework: Monte Carlo evaluation

When evaluating a given policy, we directly *bootstrapped* the estimator of $V^\pi(s_t,a_t)$, by writing $g_t^\pi = r_t + \gamma V_t(s_{t+1})$.

But an immediate way to estimate $V^\pi(s)$ is to take the empirical average of a series if realizations of the $\sum\limits_{t = 0}^\infty \gamma^t R_t$ random variable.

In plain words: simulate $\pi$ from $s$ a certain number of times to obtain trajectories, observe and accumulate the rewards along each trajectory, take the empirical average over all trajectories.

That is precisely what we did in section 5's homework to estimate the value of the starting state.

In [ ]:
%load solutions/RL1_exercise1.py

Now can we generalize this to learning the value $V^\pi$ in all states?

Let us start with a fully **offline Monte-Carlo** algorithm. The idea is simple: start from $s$, run the policy until termination (or for a long number of steps), repeat for a number of episodes, then update the value of all encountered states. This requires to store in memory full episodes (it also requires that the episodes be finite-length).

But we can immediately do better with an **online Monte-Carlo** method. It is almost the same idea: start from $s$, run the policy until termination (or for a long number of steps) then update the value of all encountered states before restarting an episode.  

Let $(s_0, r_0, s_1, \ldots, s_T)$ be the sequence of transitions of such an episode.  
For the sake of clarity we will slim down our notations: $g^\pi(s_t)$ becomes $g_t$.  
Then, this sequence provides a sample $g_t$ of $G^\pi(s_t)$ for all $s_t$ visited during the simulations. 
<div class="alert alert-success"><b>Monte Carlo return:</b>
$$g_t = \sum_{i>t} \gamma^{i-t} r_i$$
</div>

<div class="alert alert-warning">
    
**Exercise:**  
Consider the policy that always moves right in FrozenLake and implement an online Monte-Carlo estimatior for $g_t$, that you will use in a Stochastic Approximation method to obtain $V^\pi$.  
We decide we can tolerate a 0.001 error on $V^\pi$ so, to keep things simple, we set a constant $\alpha=0.001$.  
Take $\gamma = 0.9$. Run 100000 episodes of maximum 1000 time steps each.  
After each episode, compare the value function obtained with that computed with the model-based approaches of previous sections (use the cell below to recall `V_pi0` from section 6's exercises).
</div>

In [ ]:
%load solutions/RL2_exercise5.py

In [ ]:
# %load solutions/RL3_exercise1.py
### WRITE YOUR CODE HERE
# If you get stuck, uncomment the line above to load a correction in this cell (then you can execute this code).

So online Monte-Carlo allows us to update $V^\pi$ episode after episode. Some values are better estimated than others depending on how often the corresponding state was visited.

Monte-Carlo estimation has some flaws nonetheless. It still requires to store one full episode in memory before $V$ is updated. Also, one rare value for $r_t$ affects directly all the value of the states encountered before $s_t$. So we can question the robustness of this estimator.

## Homework: TD(lambda)

With Monte Carlo (MC) and TD(0), we have two methods with different features:
- TD(0): 1-sample update with bootstrapping
- MC: $\infty$-sample update with no bootstrapping

What's inbetween?
- inbetween: $n$-sample update with bootstrapping

We define the **$n$-step target** or **$n$-step return** $G^{(n)}_t$ from state $s_t$ as the random variable:

$$
\begin{array}{l|l}
G_t = R_t + \gamma R_{t+1} + \gamma^2 R_{t+2} + \ldots & \textrm{MC}\\
G^{(1)}_t = R_t + \gamma V_t(S_{t+1}) & 1\textrm{-step TD = TD(0)}\\
G^{(2)}_t = R_t + \gamma R_{t+1} + \gamma^2 V_t(S_{t+2}) & 2\textrm{-step TD}\\
G^{(n)}_t = R_t + \gamma R_{t+1} + \gamma^2 R_{t+2} + \ldots + \gamma^n V_t(S_{t+n}) & n\textrm{-step TD}
\end{array}
$$

And we define the **$n$-step TD update** as:
<div class="alert alert-success"><b>$n$-step TD update:<b>
$$V(s_t) \leftarrow V(s_t) + \alpha \left[ G^{(n)}_t - V(s_t) \right]$$
</div>

<div class="alert alert-warning">
    
**Exercise:**  
Suppose that the immediate reward $R$ has a constant variance $\sigma^2$ and that for all states $s$ the estimator $V(s)$ of $V^\pi(s)$ has bias $\epsilon$.  
What is the variance of $G_t^{(n)}$?  
What is the bias of $\mathbb{E}\left(G_t^{(n)}(s)\right)$ as an estimator of $V^\pi(s)$?  
Comment on the impact of choosing a certain value for $n$.
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>

Reminder: $var(X+Y)=var(X)+var(Y)$ and $var(aX)=a^2 var(X)$.  
Consequently:
\begin{align*}
    var(G_t^{(n)}) &= \sum_{i=0}^{n-1} \gamma^{2i} \sigma^2 + \gamma^{2n} var(V_t(S_{t+n}))\\
     &= \frac{1-\gamma^{2n}}{1-\gamma^2}\sigma^2 + \gamma^{2n} var(V_t(S_{t+n}))
\end{align*}
    
The variance grows with $n$, both because $1-\gamma^n$ grows with $n$ and because $S_{t+n}$ has larger variance as $n$ grows.
    
On the bias side:
\begin{align*}
    \mathbb{E}\left(G_t^{(n)}(s)\right) - V^\pi(s) &= \mathbb{E}\left(G_t^{(n)}(s)\right) - \mathbb{E}\left(\sum_{i=0}^\infty \gamma^t R_t\right)\\
    &=\mathbb{E}\left(\sum_{i=0}^{n-1} \gamma^i R_{t+i} + \gamma^n V_t(S_{t+n})\right)  - \mathbb{E}\left(\sum_{i=0}^\infty \gamma^t R_t\right)\\
    &=\gamma^n \left[ \mathbb{E}\left(V_t(S_{t+n})\right) - \mathbb{E}\left(\sum_{i=n}^\infty \gamma^t R_t \right) \right]\\
    &=\gamma^n \left[ \mathbb{E}\left(V_t(S_{t+n})\right) - V_t(S_{t+n}) \right]\\
    &=\gamma^n \epsilon
\end{align*}
    
So the bias decreases with $n$. This makes sense since $V_t$'s importance is weighted by $\gamma^n$.
    
Consequently, choosing a value for $n$ is making a bias-variance tradeoff. Small $n$ means small variance an large bias, large $n$ means large variance and small bias. Thus, choosing an intermediate value has in interest in accelerating the convergence of TD algorithms.
</details>

So MC corresponds to an $\infty$-step TD update.  
    
The $n$-step TD update algorithm converges to the true $V^\pi$ just as TD(0) or MC. It requires to wait for $n$ time steps before performing an update.

Remark: for finite-length  episodes of length $T$, all $n$-step returns for $n>T-t$ are equal to the Monte Carlo return $G_t$.

So $n$-step TD updates bridge a gap between MC and TD(0). But it's not quite satisfying yet because we never really know what value of $n$ is appropriate to speed up convergence for a given problem. An interesting property is that we can mix $n$ and $m$-step returns together. Consider $G^{mix}_t = \frac{1}{3} G^{(2)}_t + \frac{2}{3} G^{(4)}_t$.
Then the update $V(s_t) \leftarrow V(s_t) + \alpha \left[G^{mix}_t - V(s_t)\right]$ still converges to $V^\pi$. More generally, convex sums of $n$-step returns yield update procedures that still converge to $V^\pi$.

Now, take $\lambda\in [0,1]$ and consider the $\lambda$-return $G^\lambda_t$:
<div class="alert alert-success"><b>$\lambda$-return:</b>
$$G^\lambda_t = \left(1-\lambda\right) \sum\limits_{n=1}^\infty \lambda^{n-1}G_t^{(n)}$$
</div>

The $\lambda$-return is the mixing of *all* $n$-step returns, with weights $(1-\lambda) \lambda^{n-1}$. So, an agent performing a $\lambda$-return update looks one step in the future and uses that step to update $V(s)$ with weight $(1-\lambda)$, then looks 2 steps into the future and updates $V(s)$ with a weight $\lambda (1-\lambda)$ and so on. The illustrative figure below is an excerpt from **Reinforcement Learning: an introduction** by Sutton and Barto.

<img src="img/TD_lambda_forward.png"></img>

To get a better understanding of the $\lambda$-return and to set ideas, let us consider a finite length episode $(s_t, r_t, s_{t+1}, \ldots, s_T)$. Since the episode ends after $T$, we have $\forall k>0, \ G^{(T-t+k)}_t = G_t$. Thus, we can split the $\lambda$-return sum in two:

\begin{align*}
G^\lambda_t & = \left(1-\lambda\right) \sum\limits_{n=1}^{T-t-1} \lambda^{n-1}G_t^{(n)} + \left(1-\lambda\right) \sum\limits_{n=T-t}^{\infty} \lambda^{n-1}G_t^{(n)}\\
& = \left(1-\lambda\right) \sum\limits_{n=1}^{T-t-1} \lambda^{n-1}G_t^{(n)} + \left(1-\lambda\right) \lambda^{T-t-1} \sum\limits_{n=T-t}^{\infty} \lambda^{n-T+t} G_t^{(n)}\\
& = \left(1-\lambda\right) \sum\limits_{n=1}^{T-t-1} \lambda^{n-1}G_t^{(n)} + \left(1-\lambda\right) \lambda^{T-t-1} \sum\limits_{k=0}^{\infty} \lambda^{k} G_t^{(T-t+k)}\\
& = \left(1-\lambda\right) \sum\limits_{n=1}^{T-t-1} \lambda^{n-1}G_t^{(n)} + \lambda^{T-t-1} G_t\\
\end{align*}

So we have $G^\lambda_t = \left(1-\lambda\right) \sum\limits_{n=1}^{T-t-1} \lambda^{n-1}G_t^{(n)} + \lambda^{T-t-1} G_t$.
- When $\lambda = 0$, it is a $TD(0)$ update (hence the "0" in TD(0)).
- When $\lambda = 1$, it is a MC update.
So we can define the **$\lambda$-return algorithm** that generalizes on TD(0) and MC:
<div class="alert alert-success"><b>$\lambda$-return algorithm:</b>
$$V(s_t) \leftarrow V(s_t) + \alpha \left[G^{\lambda}_t - V(s_t)\right] $$
</div>

That's all very nice and we have replaced the choice of $n$ by the choice of $\lambda$, which seems less sensitive. But, still, we don't know how to compute those $n$-step returns, and the $\lambda$-return, without running $n$-step episodes (and thus infinite episodes for the $\lambda$-return in the general case).

This is where we need to flip the little man in the drawing above to make him look backwards in time. When an agent transitions from $s$ to $s'$ and obtains reward $r$, it can compute the $1$-step return for $s$ and perform the corresponding $1$-step TD update. Then, as it transitions from $s'$ to $s''$ and observes $r'$ it can perform the $1$-step TD update in $s'$, but also the $2$-step TD update in $s$! An so on for future transitions. So, incrementally, as time unrolls, the agent will include the $n$-step updates in the $\lambda$-return of $s$ as they become available. In the limit, when $t\rightarrow\infty$, the $\lambda$-return in every state will be complete and the agent will have completed a $\lambda$-return algorithm. This figure below (excerpt from **Reinforcement Learning: an introduction** by Sutton and Barto) illustrates this *backward-view* on TD updates.

<img src="img/TD_lambda_backward.png"></img>

This seems to imply that we need to remember the states we went through, which is quite the same as remembering full episodes for MC updates. But since we want to update a state seen $n$ steps ago with a weight $\lambda^n (1-\lambda)$, we just need to remember, for each state, the last time we visited it (and we can forget about the trajectory linking states together). This way, we store $|S|$ values at all time, instead of an increasingly long sequence of transitions. In order to do this, we introduce the notion of **eligibility trace**:
<div class="alert alert-success"><b>Eligibility trace of state $s$:</b>
$$e_t(s) = \left\{\begin{array}{ll}
\gamma \lambda e_{t-1}(s) & \textrm{if }s\neq s_t\\
1 & \textrm{if }s = s_t
\end{array}\right.$$
</div>

Initially, all states have an eligibility trace of zero. The eligibility trace of an unvisited state decays exponentially. So $e_t(s)$ measures how old the last visit of $s$ is.

Note that two alternative definitions of eligibility traces prevail:
<ul>
    <li> "<b>replacing traces</b>": $e_t(s) = 1\textrm{ if }s = s_t$
    <li> "<b>accumulating traces</b>": $e_t(s) = e_{t-1}(s) + 1\textrm{ if }s = s_t$
</ul>
Often (not always), replacing traces are used in practice.<br>
<br>
And finally we can define the TD($\lambda$) algorithm:
<div class="alert alert-success"><b>TD($\lambda$) algorithm:</b><br>
Given a new sample $(s_t,a_t,r_t,s_t')$.
<ol>
<li> Temporal difference $\delta = r_t+\gamma V(s_t') - V(s_t)$.
<li> Update eligibility traces for all states<br>
$e(s) \leftarrow \left\{\begin{array}{ll}
\gamma \lambda e(s) & \textrm{if } s\neq s_t\\
1 & \textrm{if } s=s_t
\end{array}\right.$
<li> Update all state's values $V(s) \leftarrow V(s) + \alpha e(s) \delta$
</ol>
Initially, $e(s)=0$.
</div>

Properties and remarks:
- Earlier states are given $e(s)$ *credit* for the TD error $\delta$
- If the environment contains terminal states, then $e$ should be reset to zero whenever a new trajectory begins.
- If $\lambda=0$, $e(s)=0$ except in $s_t$ $\Rightarrow$ standard TD(0)
- For $0<\lambda<1$, $e(s)$ indicates a distance $s \leftrightarrow s_t$ is in the episode.
- If $\lambda=1$, $e(s)=\gamma^\tau$ where $\tau=$ duration since last visit to $s_t$ $\Rightarrow$ MC method<br>
TD(1) implements Monte Carlo estimation on non-episodic problems!<br>
TD(1) learns incrementally for the same result as MC
- **TD($\lambda$) is equivalent to the $\lambda$-return algorithm.**
- The value of $\lambda$ can even be changed during the algorithm without impacting convergence.
- TD($\lambda$) is an on-policy algorithm: samples must be collected following the policy under evaluation.

Note that TD($\lambda$) is already a batch update (it already updates all state values) but not in the sense of SGD batches.

However, in the presentation given above, since the eligibility trace $e(s)$ is defined state by state, the formulation of TD($\lambda$) is limited to discrete state spaces and tabular function representations. The section on function approximation further down will provide an extension of TD($\lambda$) to linear function approximation.

The extension of TD($\lambda$) to the off-policy setting has been undertaken in the more general work about **[Gradient Temporal Differences](https://era.library.ualberta.ca/items/fd55edcb-ce47-4f84-84e2-be281d27b16a)** quoted earlier.

<div class="alert alert-warning"><b>Exercise:</b><br>
Implement a TD($\lambda$) algorithm to estimate $V^\pi$ fo the policy that always goes right. As before, take a constant $\alpha=0.001$, $\gamma=0.9$ and $\lambda=0.5$. Run the algorithm for 2000000 steps.
</div>

In [ ]:
# %load solutions/RL3_exercise4.py
### WRITE YOUR CODE HERE
# If you get stuck, uncomment the line above to load a correction in this cell (then you can execute this code).

## Homework: value function approximation

Often, $S$ is not finite (or is just too large to be enumerated). Consequently, $\mathcal{F}(S,\mathbb{R})$ has infinite (or just too large) dimension. Thus, tabular representations of $V$ are not possible and one needs to turn to function representations $V_\theta$ or $Q_\theta$ with parameters $\theta$. In this section, we provide a very short introduction to approximation methods for $V$ and $Q$.

The FrozenLake example is a toy problem with very few states (moreover discrete). It does not lend itself to a convincing demonstration of value function approximation. We shall remain at the theoretical level for the following considerations and reserve practice for later.

**Linear value function approximation**

Suppose we write $V$ as a linear model:
$$V(s) = \theta^T \varphi(s) = \sum_{i=1}^K \theta_i \varphi_i(s)$$

We wish to approximate $V(s)$ as a linear combination of features $\varphi(s)=\left(\varphi_i(s)\right)_{i\in[1,K]}$. This way, $V$ lives in the $K$-dimensional function space $span(\varphi)$. We have plenty of families of functions that we can rely on and the user's expertise plays a big role in choosing a proper **functional basis**. Generally speaking, we would expect the following properties from a good basis:
- the target $V^\pi$ can be closely approximated by its projection on $\varphi$
- given an initial $V_0 \in span(\varphi)$ and the recurrence relation $V_{n+1} = \Pi_\varphi (T^\pi V_n)$ (where $\Pi_\varphi$ is the projection operator on $span(\varphi)$), $V_n$ should be a "close enough" approximation of $T^\pi V_n$. This property is illustrated by the figure below with $Q$ instead of $V$ - excerpt from **[Least-Squares Policy Iteration](https://www.jmlr.org/papers/v4/lagoudakis03a.html)** by M. G. Lagoudakis and R. Parr (2003).
- $\varphi$ should form a basis (that is $\varphi_i \bot \varphi_j$)

<img src="img/projection.png" style="width: 600px;"></img>

If $\sum_{i=1}^K \varphi_i(s) = 1$, then $V_\theta$ is called an *averager*. Averagers are known to be well-behaved for iterative function approximation. Otherwise, other non-averager families of functions are commonly used:

- $\cos$, $\sin$ over state variables (mimics the Fourier transform, extends to wavelet bases)
- polynomials of the state variables (mimics the Taylor expansion)
- radial basis functions of the state variables (performs local approximation, extends to kernel smoothing).
- among averagers, piecewise constant local functions $\varphi_i(s) \in \{0;1\}$ group *neighborhoods* in the state space together (note the similarity with tree-based regressors).

A very straightforward way of building feature sets is to define features depending on a single state variable and then using the tensor product in order to obtain all possible combinations of sigle-variable features. More formally and more generally, suppose $S \subset S_1\times \ldots \times S_k$ and suppose $\varphi^{(i)}$ defines $d_i$ features over $S_k$; then the tensor product $\varphi^{(1)} \otimes \ldots \otimes \varphi^{(k)}$ yields $d=d_1\ldots d_k$ feature functions on $S$. But there is a catch, the number of these resulting features grows exponentially with $k$ and so does the dimension of the value function's search space $span(\varphi)$: that is the **curse of dimensionality** that makes searching for a value function exponentially more difficult as the state space dimension grows.

Additionnaly, there is **no guarantee** that, for a given $V_n \in span(\varphi)$, $T^\pi V_n$ actually lives in $span(\varphi)$.

But on the bright side, given an initial state $s_0$, the actual reachable space $S'$ given $\pi$ might be much smaller than $S$. So, in practice, we just need to obtain a good approximation of $V$ on the subspace $S'$.

Anyway, to conclude this short paragraph on feature engineering:
- good feature engineering in RL is even more crucial than in supervised learning.
- it can be very problem-dependent.
- good function approximators (generally non-parametric to avoid the fixed $span(\varphi)$) are of crucial importance.
We will discuss non-linear and non-parametric function approximation a bit further in the notebook.

Linear function approximation has played a major role in the RL literature, in particular for temporal differences methods. The **[Policy Evaluation with Temporal Differences](https://www.jmlr.org/papers/v15/dann14a.html)** survey by C. Dann et al. (2014) provides a great overview of this literature.

**The tabular case is just a specific case of linear approximation**

<div class="alert alert-warning">

**Exercise:**  
If the sentence above is true, what is the set of basis functions corresponding to a tabular representation of $V$?
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>

In the discrete state space case, consider the averager defined as:
$$\varphi_i(s) = \left\{\begin{array}{ll}1 & \textrm{if }s=s_i\\ 0 & \textrm{otherwise}\end{array}\right.$$
Feature function $\varphi_i$ is the indicator function of state $s_i$. Therefore, we have $|S|$ feature functions. So when we write $V(s) = \sum_{i=1}^{|S|} \theta_i \varphi_i(s)$, we actually have $V(s_i) = \theta_i$. Therefore the tabular representation of $V$ is equivalent to a linear model with the $\varphi_i$ feature functions.
</details>

Based on the previous exercise, let us rewrite TD($\lambda$) as a linear model update (we take the accumulating traces version; the replacing traces case is equivalent). We had previously:<br>
Given a new sample $(s_t,a_t,r_t,s_t')$.
<ol>
<li> Temporal difference $\delta = r_t+\gamma V(s_t') - V(s_t)$.
<li> Update eligibility traces for all states<br>
$e(s) \leftarrow \left\{\begin{array}{ll}
\gamma \lambda e(s) & \textrm{if } s\neq s_t\\
1 + \gamma \lambda e(s)& \textrm{if } s=s_t
\end{array}\right.$
<li> Update all state's values $V(s) \leftarrow V(s) + \alpha e(s) \delta$
</ol>
Initially, $e(s)=0$.

The temporal difference can be rewritten $\delta = r_t+\gamma\theta^T \varphi(s_t') - \theta^T \varphi(s_t)$.

The eligibility trace update can be rewritten $e \leftarrow \varphi(s) + \gamma \lambda e$.

Similarly the value update can be rewritten $\theta \leftarrow \theta + \alpha e \delta$.

Remark:  
Recall the discussion on the importance of the behavior distribution? We concluded that tabular representations were a specific case where TD(0) is truly off-policy because the influence of a sample was limited to the value of $Q$ in the corresponding $(s,a)$ pair. This discussion can be generalized for averagers (although it remains an approximation): such local models are well-behaved to suffer less from the shift in behavior distributions.

**TD($\lambda$) as a linear approximation update**

We generalize the previous result to the general linear model case:
<div class="alert alert-success"><b>TD($\lambda$) with linear function approximation:</b><br>
With $V(s) = \sum_{i=1}^K \theta_i \varphi_i(s)$, $e \in \mathbb{R}^K$.<br>
Initially, $e=0$.<br>
Given a new sample $(s_t,a_t,r_t,s_t')$.
<ol>
<li> Temporal difference $\delta = r_t+\gamma\theta^T \varphi(s_t') - \theta^T \varphi(s_t)$.
<li> Update eligibility traces for all states $e \leftarrow \varphi(s) + \gamma \lambda e$
<li> Update value function $\theta \leftarrow \theta + \alpha e \delta$
</ol>
</div>

Note that we have provided the results above without proof. We will admit them and refer the reader to RL textbooks for a rigorous justification.

Further reading on TD($\lambda$) with linear function approximation: 
**[True Online TD($\lambda$)](http://proceedings.mlr.press/v32/seijen14.html)** by H. Van Seijen and R. Sutton (2014).  
An unpublished negative result that somehow follows from this article is also that in the general case it is not possible to have a TD($\lambda$) algorithm performing on non-linear function approximation and being equivalent to the $\lambda$-return algorithm.

**Non-parametric models**

Non-parametric models generally refer to function approximators that do not rely on an a-priori fixed finite-dimensional search space and allows the representation space to evolve as needed. Among those non-parametrics models, one can count:
- linear approximations that incrementally enrich the functional basis (e.g. **[this article](https://dl.acm.org/doi/abs/10.1145/1390156.1390251)**).
- general supervised learning methods: SVMs, k-nearest neighbours (kernel smoothing methods), Gaussian Processes, tree-based methods, neural networks, etc.
  In this second category, it is generally useful to distinguish between
  - methods that explicitly minimize the L2 loss defined earlier (e.g. neural networks),
  - methods that minimize some other loss (e.g. random forests) and provide alternate (better or worse guarantees).

One quickly realizes that the frontier between parametric and non-parametric models is blur. In the general case of a $s \mapsto V(s)$ function approximator, the general idea is to feed this approximator with samples of the form $(s, r+\gamma V(s'))$. But beware: most of the nice results are generally lost when one leaves the realm of linear function approximation. More precisely, when one combines **function approximation**, **off-policy learning** and **bootstrapping** in a temporal difference method, all results are generally lost. This has been studied as the **[Deadly triad of Reinforcement Learning](https://arxiv.org/abs/1812.02648v1)** (H. Van Hasselt et al., 2018).

## Homework: Generalized Policy Iteration and Actor-Critic architectures

Remember how we went from Policy Iteration (figure below) to Asynchronous Policy Iteration?
<img src="img/policyiteration.png"></img>

Now recall the principle of Asynchronous Policy Iteration. If we consider the two elementary operations:
- Bellman backup on $Q$: $Q(s,a) \leftarrow r(s,a) + \gamma \sum\limits_{s'} p(s'|s,a) Q(s',\pi(s'))$
- Bellman backup on $\pi$: $\pi(s) \leftarrow \arg\max_{a} Q(s,a)$

Then, as long as every state and every action is visited infinitely often for Bellman backups on $Q$ or $\pi$, the sequences of $Q_n$ and $\pi_n$ converge to $Q^*$ and $\pi^*$.

Value Iteration: in each state, one update of $Q$ and one improvement of $\pi$.<br>
Policy Iteration: update $Q$ in all states until convergence, then update $\pi$ in all states.

**Generalized Policy Iteration** is the case where one has two interacting processes: policy evaluation and policy improvement, directly from samples (not from the model anymore). If these processes converge to their respective targets, then Generalized Policy Iteration converges to $Q^*$ and $\pi^*$. Model-free policy evaluation can take many forms: indirect RL, Monte Carlo evaluations, TD methods...

This leads to the definition of the general **actor-critic architectures** (excerpt from [**Algorithms for Reinforcement Learning**](https://sites.ualberta.ca/~szepesva/rlbook.html) by C. Szepesvari):

<img src="img/actor-critic.png"  style="width: 500px;"></img>

In such architectures, an *actor* chooses an action based on the current state and the information provided by the *critic*, while the *critic* constantly aims at learning relevant things in order to help the *actor* decide (value functions for example, or an approximate model).

Almost all Reinforcement Learning algorithms fall into an actor-critic architecture.

## Homework: SARSA

<div class="alert alert-warning">

**Exercise:**  
Suppose we modify TD(0) on $Q$-functions so that, instead of picking a random action at each time step, we pick the greedy action with respect to $Q$. What is the risk in doing this?
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>

A deterministic, greedy policy will not try all actions in all states infinitely often. So we can't guarantee convergence, neither of $Q$ to any policy's $Q^\pi$ (because if some states are not visited no updates will take place for them), nor of the $Q$-greedy policy to $\pi^*$. The conditions for the convergence of Generalized Policy Iteration are just not met.
</details>

To solve this issue we introduce the notion of **Greedy in the limit of infinite exploration** (GLIE) actor:  

<div class="alert alert-success"><b>Greedy in the limit of infinite exploration</b> (GLIE) actor:<br>
A GLIE actor guarantees that:<br>
- All state-action pairs $(s,a)$ are visited infinitely often for updates of $Q$, as $t\rightarrow\infty$:
$$\lim_{t\rightarrow\infty} count_t(s,a) = \infty$$
- As $t\rightarrow\infty$ the actor becomes $Q$-greedy, that is:
$$\lim_{t\rightarrow\infty} \pi_t(a|s) = \mathbb{1}\left(a = \arg\max_{\hat{a}} Q(s,\hat{a})\right)$$
</div>

An example of GLIE actor is the so-called $\epsilon$-greedy exploration strategy (that we introduced with Q Learning) that uniformly picks a non-greedy action with probability $\epsilon$:
$$\pi_t(a|s) = \left\{\begin{array}{ll}1-\epsilon_t & \textrm{if }a=\arg\max_{\hat{a}} Q(s,\hat{a})\\
\frac{\epsilon_t}{|A|-1} & \textrm{otherwise} \end{array}\right.$$
With a parameter $\epsilon_t>0$ that goes to zero as $t$ tends to $\infty$, one obtains a GLIE actor.

GLIE actors (or policies) enforce the limits of the **exploration vs. exploitation tradeoff**. As long as actors respect the GLIE properties, actor-critic architectures fall within the convergence properties of Generalized Policy Iteration.

With this last definition, we have all the ingredients to define algorithms that evolve to an optimal behaviour.

**A remark**

You might have noticed that the totally random policy we applied in the TD(0) update was a very naive choice. Even if all states are reachable from anywhere in the state space (ergodicity property), they might not all be visited with the same frequency and therefore the convergence to $Q^\pi$ might be delayed because of this uniform exploration strategy.  

The same remark applies to $\epsilon$-greedy exploration strategies that often start with $\epsilon=1$ and let it slowly decrease towards zero. These strategies don't account for the actual *visitation frequencies* of state-action pairs. In some states it might be good to keep a strong exploration probability because they actually have been seldom visited, while in other states, a faster decrease is desirable. This links also to the question of *value propagation* that was underpinned by the asynchronous value iteration remarks in section 6.  

The point here is to notice that $\epsilon$-greedy is a simple, very naive exploration strategy that fits within the GLIE requirements but that much better exploration policies are possible by taking the current state into account (contextual exploration) or by using the values of the temporal difference (prioritized experience replay) and the $Q$ estimate (Boltzmann policies, $E^3$ or $R_{max}$ strategies) for instance.

**SARSA**

The key idea behind the SARSA algorithm is to build a critic that constantly tries to evaluate the value $Q$ of the actor's policy $\pi$, and an actor that tends to be greedy with respect to this critic. The algorithm is written:

<div class="alert alert-success">

**SARSA**  
In $s$, choose (*actor*) $a$ using $Q$, then repeat:
<ol>
<li> Observe $r$, $s'$
<li> Choose $a'$ (<i>GLIE actor</i>) using $Q$
<li> Temporal difference: $\delta=r+\gamma Q(s',a') - Q(s,a)$
<li> Update $Q$: $Q(s,a) \leftarrow Q(s,a) + \alpha \delta$
<li> $s\leftarrow s'$, $a\leftarrow a'$
</ol>
SARSA converges if the actor is GLIE and if $\alpha$ respects the Robbins-Monro conditions.
</div>

It is important to note that SARSA is an **on-policy** critic: it constantly evaluates the current $\pi$... that constantly shifts towards $\pi^*$ by being $Q$-greedy.

The name SARSA comes from the usage of an augmented sample $(s,a,r,s',a')$.<br>
<br>
<div class="alert alert-warning"><b>Exercice:</b><br>
Let's implement an $\epsilon$-greedy SARSA on the FrozenLake problem.<br>
For the decrease of $\epsilon$ we can opt for a division by 2 every million steps.<br>
Keep track of the state-action visitation count, as in the Q-learning example.<br>
You can compare with $Q^*$ and $\pi^*$ obtained during the model-based class.<br>
The utility functions below are here to make the task easier.
</div>

In [ ]:
# Model-based policy optimization and a few utility functions

def value_iteration(V,epsilon,max_iter):
    W = np.copy(V)
    residuals = np.zeros((max_iter))
    for i in range(max_iter):
        for s in range(env.observation_space.n):
            Q = np.zeros((env.action_space.n))
            for a in range(env.action_space.n):
                outcomes = env.unwrapped.P[s][a]
                for o in outcomes:
                    p  = o[0]
                    s2 = o[1]
                    r  = o[2]
                    Q[a] += p*(r+gamma*V[s2])
            W[s] = np.max(Q)
            #print(W[s])
        residuals[i] = np.max(np.abs(W-V))
        #print("abs", np.abs(W-V))
        np.copyto(V,W)
        if residuals[i]<epsilon:
            residuals = residuals[:i+1]
            break
    return V, residuals

def Q_from_V(V):
    Q = np.zeros((env.observation_space.n, env.action_space.n))
    for s in range(env.observation_space.n):
        for a in range(env.action_space.n):
            outcomes = env.unwrapped.P[s][a]
            for o in outcomes:
                p  = o[0]
                s2 = o[1]
                r  = o[2]
                Q[s,a] += p*(r+gamma*V[s2])
    return Q

def greedyQpolicy(Q):
    pi = np.zeros((env.observation_space.n),dtype=np.int)
    for s in range(env.observation_space.n):
        pi[s] = np.argmax(Q[s,:])
    return pi

def to_s(row,col):
    return row*env.unwrapped.ncol+col

def to_row_col(s):
    col = s%env.unwrapped.ncol
    row = int((s-col)/env.unwrapped.ncol)
    return row,col

def print_policy(pi):
    for row in range(env.unwrapped.nrow):
        for col in range(env.unwrapped.ncol):
            print(actions[pi[to_s(row,col)]], end='')
        print()
    return

Vinit = np.zeros((env.observation_space.n))
Vstar,residuals = value_iteration(Vinit,1e-4,1000)
Qstar = Q_from_V(Vstar)
print(actions)
print(Vstar)
print(Qstar)
pi_star = greedyQpolicy(Qstar)
print_policy(pi_star)
env.render();

In [ ]:
# %load solutions/RL4_exercise4.py
### WRITE YOUR CODE HERE
# If you get stuck, uncomment the line above to load a correction in this cell (then you can execute this code).

In [ ]:
count_map = np.zeros((env.unwrapped.nrow, env.unwrapped.ncol, env.action_space.n))
for a in range(env.action_space.n):
    for x in range(env.observation_space.n):
        row,col = to_row_col(x)
        count_map[row, col, a] = count[x,a]

fig, axs = plt.subplots(ncols=4)
for a in range(env.action_space.n):
    name = "a = " + actions[a]
    axs[a].set_title(name)
    axs[a].imshow(np.log(count_map[:,:,a]+1), interpolation='nearest')
    #print("a=", a, ":", sep='')
    #print(count_map[:,:,a])
plt.show()
env.render()

The same remark as for TD(0) holds: with a better exploration strategy, the convergence to Qstar could be much more efficient. We can see that some states have been visited (and updated) way more often than others.

<div class="alert alert-warning"><b>Exercices (open questions, no solutions provided):</b><br>
<ul>
<li><b>Context-dependent exploration</b><br>
Can you implement an $(s,a)$-dependent $\epsilon$-greedy exploration strategy (by using the `count` table introduced earlier for instance)?
<li><b>Heuristic initialization on $Q$</b><br>
For Q-learning, can you think of an initialization of $Q$ that would be better than plain zeros (for example by exploiting the maximum 1-step reward $r_{max}$)? One that, for instance, would drive the exploration towards unvisited states?
<li><b>Reward shaping</b><br>
Did you notice that falling into a hole brings no penalty? If we introduced a $-1$ reward for falling into a hole, would it change the optimal policy?
<li><b>SARSA($\lambda$)</b><br>
SARSA is an on-policy method and we've seen that so is TD($\lambda$) so it seems rather straightforward to implement a SARSA($\lambda$) algorithm that, hopefully, will have better convergence properties than plain SARSA.
<li><b>$Q(\lambda)$</b><br>
This time it is not as straightfoward to derive a $Q(\lambda)$ algorithm from TD($\lambda$), precisely because TD($\lambda$) evaluates the policy being applied and not another one. Can you imagine a way to still perform $Q(\lambda)$ updates? An answer is found in Watkins's thesis that introduces Q-learning in 1989. For a more recent approach and other references, see Sutton et al, <b>A new Q($\lambda$) with interim forward view and Monte Carlo equivalence</b>, 2014)
<li><b>SARSA and $Q$-learning with linear value function approximation</b><br>
Can you implement an approximate version of SARSA and $Q$-learning with linear Q-function approximation $Q(s,a)=\theta^T\varphi(s,a)$?
</ul>
</div>